In [2]:
import numpy as np
import os
from google.colab import drive

In [3]:
drive.mount('/content/drive')

directory_path = '/content/drive/My Drive/NLP/clips__test'

print(len(os.listdir(directory_path)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
20


In [ ]:
def segment_npy_files(source_directory, target_directory, segment_length, overlap=0):
    """
    Segments the .npy files stored in source_directory and saves the segments to target_directory.

    Parameters:
    - source_directory: Path to the directory containing the .npy files.
    - target_directory: Path where segmented .npy files will be saved.
    - segment_length: Number of frames in each segment.
    - overlap: Number of frames that consecutive segments will overlap.
    """
    # Ensure the target directory exists
    os.makedirs(target_directory, exist_ok=True)

    # List all .npy files in the source directory
    for filename in os.listdir(source_directory):
        if filename.endswith(".npy"):
            file_path = os.path.join(source_directory, filename)
            data = np.load(file_path)

            # Calculate the number of segments to create
            step_size = segment_length - overlap
            num_segments = (data.shape[0] - overlap) // step_size

            # Generate segments
            segments = [
                data[i * step_size: i * step_size + segment_length]
                for i in range(num_segments)
                if i * step_size + segment_length <= data.shape[0]
            ]

            # Save each segment
            for idx, segment in enumerate(segments):
                segment_filename = f"{os.path.splitext(filename)[0]}_seg{idx}.npy"
                segment_path = os.path.join(target_directory, segment_filename)
                np.save(segment_path, segment)

source_dir = '/path/to/source/npy/files'
target_dir = '/path/to/save/segments'
segment_len = 100

segment_npy_files(source_dir, target_dir, segment_len)